In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    recs = np.random.choice(items_weights['item_id'], size=n, replace=False, p=items_weights['weight'])
    
    return recs.tolist()
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    return recs.tolist()

In [5]:
%%time

# your_code

weighted_train = data_train.groupby('item_id')['quantity'].sum().reset_index()
weighted_train.sort_values('quantity', ascending=False, inplace=True)
weighted_train['weight'] = np.log10(weighted_train['quantity'] / weighted_train['quantity'].sum() + 1)
weighted_train['weight'] = weighted_train['weight'] / weighted_train['weight'].sum()
weighted_train

result_test = data_test.groupby('user_id')['item_id'].unique().reset_index()
result_test.columns = ['user_id', 'actual']

result_test['w_random'] = result_test['user_id'].apply(lambda x: weighted_random_recommendation(weighted_train))

Wall time: 4.62 s


In [6]:
result_test.head(3)

,user_id,actual,w_random
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6534178, 6533889, 6534166, 480014, 1404121]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6533889, 6534178, 480014, 6534166, 1404121]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[6534178, 6534166, 6533889, 5668996, 6544236]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [7]:
result = pd.read_csv('predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[1019719, 2332987, 5550112, 9858801, 1072575]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[839002, 13158305, 12354593, 8199063, 15597006]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]"


In [8]:
result.actual = result.actual.replace('[ ]+', ',', regex=True).replace('\[,', '[', regex=True).replace('\],', ']', regex=True)
clm = result.columns.to_list()
clm.remove('user_id')

for c in clm:
    result[c] = result[c].apply(lambda x: eval(x))

clm.remove('actual') # для будущих вычислений

result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1019719, 2332987, 5550112, 9858801, 1072575]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[839002, 13158305, 12354593, 8199063, 15597006]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]"


In [9]:
# your_code

def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    
    if len(relevant_indexes) == 0:
        return 0
    
    amount_relevant = len(relevant_indexes)
    
    sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant+1) for index_relevant in relevant_indexes])
    return sum_/amount_relevant

def map_k(recommended_list, bought_list, k=5):
    
    result = []
    for i in range(len(bought_list)):
        result.append(ap_k(recommended_list[i], bought_list[i], k))
    
    result = sum(result) / len(bought_list)
    
    return result

In [10]:
for c in clm:
    print(f'{c:30}: {round(map_k(result[c], result["actual"]), 5)}')

random_recommendation         : 0.00169
popular_recommendation        : 0.25681
itemitem                      : 0.22094
cosine                        : 0.37428
tfidf                         : 0.3816
own_purchases                 : 0.42562


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров

In [11]:
# your_code
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index().sort_values('quantity', ascending=False)
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top = popularity.item_id.to_list()

In [15]:
res = dict()
for i in range(5, 31, 5):
    copy_w_train = weighted_train.copy()
    copy_w_train.loc[~copy_w_train['item_id'].isin(top[:i * 1000]), 'item_id'] = 999999
    copy_w_train = copy_w_train.groupby('item_id')['quantity'].sum().reset_index()
    copy_w_train['weight'] = np.log10(copy_w_train['quantity'] / copy_w_train['quantity'].sum() + 1)
    copy_w_train['weight'] = copy_w_train['weight'] / copy_w_train['weight'].sum()
    
    result_test['w_i_random'] = result_test['user_id'].apply(lambda x: weighted_random_recommendation(copy_w_train))
    res[i * 1000] = map_k(result_test.w_i_random, result_test.actual)

res

{5000: 0.18403525954946123,
 10000: 0.1840270976167156,
 15000: 0.18455354227881154,
 20000: 0.1849194689302426,
 25000: 0.18307215148547162,
 30000: 0.18686200892371307}

- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [16]:
top_data_train = data_train.copy()
top_data_train.loc[~top_data_train.item_id.isin(top[:5000]), 'item_id'] = 999999

user_item_matrix_train = pd.pivot_table(top_data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

userids = user_item_matrix_train.index.values
itemids = user_item_matrix_train.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [17]:
user_item_matrix_train = user_item_matrix_train.astype('float')
sparse_user_item_train = csr_matrix(user_item_matrix_train).tocsr()
sparse_user_item_train_T = csr_matrix(user_item_matrix_train).T.tocsr()

In [35]:
res = dict()
for k in range(1, 20):
    model = ItemItemRecommender(K=k, num_threads=-1)

    model.fit(sparse_user_item_train_T,  # На вход item-user matrix
              show_progress=False)

    result_test['itemitem'] = result_test['user_id'].apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                                                                      model.recommend(userid=userid_to_id[x],
                                                                                      user_items=sparse_user_item_train,   # на вход user-item matrix
                                                                                      N=5,
                                                                                      filter_already_liked_items=False,
                                                                                      filter_items=None,
                                                                                      recalculate_user=False)])

    res[k] = map_k(result_test.itemitem, result_test.actual)
res

{1: 0.0,
 2: 0.3092148220698674,
 3: 0.2886229459135928,
 4: 0.2667224398737619,
 5: 0.24875802590053253,
 6: 0.24875802590053253,
 7: 0.24875802590053253,
 8: 0.24875802590053253,
 9: 0.24875802590053253,
 10: 0.24875802590053253,
 11: 0.24875802590053253,
 12: 0.24875802590053253,
 13: 0.24875802590053253,
 14: 0.24875802590053253,
 15: 0.24875802590053253,
 16: 0.24875802590053253,
 17: 0.24875802590053253,
 18: 0.24875802590053253,
 19: 0.24875802590053253}

Начиная с 5 соседей метрика никак не меняется, до этого значения наблюдаются значения повыше.